# Keras Tutorail - the Happy House

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [16]:
from keras.models import Sequential

## 1. Dataset

In [2]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


## 2. Building model in Keras

In [3]:
def happy_model(input_shape):
    x_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(x_input)
    
    X = Conv2D(32, (7,7), strides=(1, 1),name = "conv0")(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    X = Conv2D(32, (7,7), strides=(1, 1),name = "conv1")(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(inputs=x_input,outputs=X,name="HappyModel")
    
    return model

In [20]:
batch_size = 32
num_classes = 2
epochs = 5

In [22]:
import keras

In [24]:
y_train = keras.utils.to_categorical(Y_train, num_classes)
y_test = keras.utils.to_categorical(Y_test, num_classes)

In [34]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))# a.k.a. conv2d_1
# 32 filters(kernels) of shape 3*3 
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))# a.k.a. conv2d_2
# 32 filters(kernels) of shape 3*3 
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2))) # a.k.a. max_pooling2d_1
# Downsampling 
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))# a.k.a. conv2d_3 
# 64 filters(kernels) of shape 3*3
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))# a.k.a. conv2d_4
# 64 filters(kernels) of shape 3*3 
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
# Downsampling 
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_29 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_30 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 31, 31, 64)        18496     
__________

In [36]:
model.fit(x=X_train,y=Y_train,batch_size=32,epochs=5)

Epoch 1/5
600/600 [==============================] - 23s 38ms/step - loss: 7.9712 - acc: 0.5000
Epoch 2/5
600/600 [==============================] - 20s 33ms/step - loss: 7.9712 - acc: 0.5000
Epoch 3/5
600/600 [==============================] - 20s 34ms/step - loss: 7.9712 - acc: 0.5000
Epoch 4/5
600/600 [==============================] - 21s 36ms/step - loss: 7.9712 - acc: 0.5000
Epoch 5/5
600/600 [==============================] - 21s 36ms/step - loss: 7.9712 - acc: 0.5000


## 3. Train & test

In [5]:
happyModel = happy_model(X_train.shape[1:])

In [6]:
happyModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [8]:
happyModel.fit(x=X_train,y=Y_train,batch_size=32,epochs=50)

Epoch 1/50
600/600 [==============================] - 27s 46ms/step - loss: 2.1654 - acc: 0.5800
Epoch 2/50
600/600 [==============================] - 27s 45ms/step - loss: 0.3109 - acc: 0.8633
Epoch 3/50
600/600 [==============================] - 27s 46ms/step - loss: 0.1622 - acc: 0.9433
Epoch 4/50
600/600 [==============================] - 27s 46ms/step - loss: 0.1449 - acc: 0.9433
Epoch 5/50
600/600 [==============================] - 26s 44ms/step - loss: 0.1072 - acc: 0.9683
Epoch 6/50
600/600 [==============================] - 27s 45ms/step - loss: 0.1009 - acc: 0.9667
Epoch 7/50
600/600 [==============================] - 27s 44ms/step - loss: 0.1197 - acc: 0.9583
Epoch 8/50
600/600 [==============================] - 27s 44ms/step - loss: 0.0989 - acc: 0.9617
Epoch 9/50
600/600 [==============================] - 28s 47ms/step - loss: 0.1047 - acc: 0.9633
Epoch 10/50
600/600 [==============================] - 28s 47ms/step - loss: 0.0900 - acc: 0.9717
Epoch 11/50
600/600 [========

In [9]:
preds = happyModel.evaluate(X_test,Y_test,batch_size=32,verbose=1,sample_weight=None)

print("Loss = ",str(preds[0]))
print("Test accuracy = ",str(preds[1]))

150/150 [==============================] - 3s 21ms/step
Loss =  1.368675799369812
Test accuracy =  0.673333334128062


## 4. Model debug

In [10]:
happyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
__________

### Calculating number of weights for Con layer
-------------------------
[Calculate # of weights](https://stackoverflow.com/questions/44608552/keras-cnn-model-parameters-calculation)

    - In this formula '+1' is because of bias for per filter

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))